In [22]:
import os
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


In [23]:

# Path ke driver Selenium dan file cookie
# driver_path = '/path/to/chromedriver'  # Ganti dengan path driver Anda
cookie_path = 'youtube_cookies.json'
profile_path = 'C:\\Users\\LENOVO\\AppData\\Local\\Google\\Chrome\\User Data\\Default'  # Ganti path ke profil Chrome Anda



In [24]:

def save_cookies(driver, path):
    """Menyimpan cookie dari Selenium WebDriver ke file JSON."""
    with open(path, 'w') as file:
        json.dump(driver.get_cookies(), file)

def load_cookies(driver, path):
    """Memuat cookie dari file JSON ke Selenium WebDriver."""
    if os.path.exists(path):
        with open(path, 'r') as file:
            cookies = json.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)

def login_youtube(driver):
    """Login manual ke YouTube dan simpan cookie setelahnya."""
    driver.get("https://www.youtube.com")
    input("Silakan login di YouTube dan tekan Enter setelah selesai...")  # Menunggu pengguna login manual
    save_cookies(driver, cookie_path)

def initialize_driver_with_profile(profile_path):
    """Inisialisasi driver dengan profil Chrome yang sudah ada."""
    chrome_options = Options()
    chrome_options.add_argument(f"user-data-dir={profile_path}")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # service = Service(driver_path)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def get_youtube_comments(driver, video_url, max_comments=100):
    """Mengambil komentar dari video YouTube dengan sesi login."""
    driver.get(video_url)
    time.sleep(3)  # Tunggu sampai halaman dimuat

    comments = []
    count = 0

    while count < max_comments:
        driver.execute_script("window.scrollBy(0, 300);")  # Scroll untuk memuat lebih banyak komentar
        time.sleep(1)

        comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        for element in comment_elements[count:max_comments]:
            comment_text = element.text
            if comment_text:
                comments.append(comment_text)
                count += 1
            if count >= max_comments:
                break

    return comments

def save_to_csv(comments, filename="youtube_comments.csv"):
    """Menyimpan komentar ke file CSV."""
    df = pd.DataFrame(comments, columns=["Comment"])
    df.to_csv(filename, index=False)
    print(f"Komentar berhasil disimpan di {filename}")


In [25]:

# Inisialisasi WebDriver dengan profil Chrome
driver = initialize_driver_with_profile(profile_path)


In [26]:

# Cek jika cookie sudah ada; jika tidak, lakukan login manual
if os.path.exists(cookie_path):
    driver.get("https://www.youtube.com")
    load_cookies(driver, cookie_path)
    driver.refresh()  # Refresh untuk menerapkan cookie
else:
    login_youtube(driver)  # Login manual dan simpan cookie

# Ambil komentar dari video
video_url = "https://www.youtube.com/watch?v=i6IOiUi6IYY"  # Ganti dengan ID video yang diinginkan
comments = get_youtube_comments(driver, video_url, max_comments=100)
save_to_csv(comments)

# Tutup driver setelah selesai
driver.quit()


Komentar berhasil disimpan di youtube_comments.csv
